In [18]:
import pandas as pd 
import numpy as np 
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [19]:
#1. Đọc dữ liệu
train_df = pd.read_csv(r"D:\Năm 3 - HK2\Phân tích dữ liệu kinh doanh\IS403.P23.OnlineSalesData\Dataset\712\train.csv", parse_dates=['InvoiceDate'])
val_df = pd.read_csv(r"D:\Năm 3 - HK2\Phân tích dữ liệu kinh doanh\IS403.P23.OnlineSalesData\Dataset\712\valid.csv", parse_dates=['InvoiceDate'])
test_df = pd.read_csv(r"D:\Năm 3 - HK2\Phân tích dữ liệu kinh doanh\IS403.P23.OnlineSalesData\Dataset\712\test.csv", parse_dates=['InvoiceDate'])

In [20]:
#2. Chuẩn bị dữ liệu 
def preprocess(df):
    df = df.copy()
    df['OrderTotal'] = (df['Quantity'] * df['UnitPrice'] * (1-df['Discount']) + df['ShippingCost'])
    df = df[['InvoiceDate', 'OrderTotal']]
    df = df.set_index('InvoiceDate').resample('D').sum().reset_index()
    df['dayofweek'] = df['InvoiceDate'].dt.dayofweek
    df['month'] = df['InvoiceDate'].dt.month
    df['day'] = df['InvoiceDate'].dt.day
    df['year'] = df['InvoiceDate'].dt.year
    df['lag1'] = df['OrderTotal'].shift(1)
    df['lag7'] = df['OrderTotal'].shift(7)
    df.dropna(inplace=True)
    return df

train = preprocess(train_df)
valid = preprocess(val_df)
test = preprocess(test_df)


In [21]:
# 3. Tạo tập dữ liệu huấn luyện & kiểm thử
X_train, y_train = train[['dayofweek','month','day','year','lag1','lag7']], train['OrderTotal']
X_valid, y_valid = valid[['dayofweek','month','day','year','lag1','lag7']], valid['OrderTotal']
X_test, y_test   = test[['dayofweek','month','day','year','lag1','lag7']], test['OrderTotal']

In [22]:
# 4. Huấn luyện mô hình
model = XGBRegressor(n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train, 
          eval_set=[(X_valid, y_valid)], 
          early_stopping_rounds=10, verbose=False)


TypeError: XGBModel.fit() got an unexpected keyword argument 'early_stopping_rounds'